In [14]:
import math
import pandas as pd
from scipy import interpolate

In [15]:
athm_pr = 100000                           # [Pa]
dm = 3.8E-10                               # [-]
R = 8.314462                               # [J/(mol - K)]
M = 1.6E-2                                 # [kg/mol]
Na = 6.0221415E23                          # [1/mol]
b = -1
dpdl = 0.1                                 # [MPa/m]

In [16]:
properties = pd.read_csv('z,cg,n.csv')
p = properties['P'].values*1000000
t = properties['T'].values
f_z = interpolate.bisplrep(p, t, properties['Z'].values, s=2)
f_rho = interpolate.bisplrep(p, t, properties['Density(kg/m3)'].values, s=2)
f_vis = interpolate.bisplrep(p, t, properties['Viscosity(uPa-s)'].values/1000000, s=2)
f_cg = interpolate.interp1d(p, properties['cg(1/Mpa)'].values/1000000)

In [17]:
var_ranges = pd.read_csv('variability_ranges.csv').set_index('Variable')

p = (var_ranges['Min']['Pressure']+var_ranges['Max']['Pressure'])/2  # [Pa]
por = (var_ranges['Min']['Porosity']+var_ranges['Max']['Porosity'])/2       # [-]
ro = (var_ranges['Min']['Radius']+var_ranges['Max']['Radius'])/2            # [m]
tor = (var_ranges['Min']['Tortuosity']+var_ranges['Max']['Tortuosity'])/2   # [-]
op = (var_ranges['Min']['Overburden Pressure']+var_ranges['Max']['Overburden Pressure'])/2      # [Pa]
q = (var_ranges['Min']['q']+var_ranges['Max']['q'])/2   # [-]
t = (var_ranges['Min']['t']+var_ranges['Max']['t'])/2   # [-]
k = (var_ranges['Min']['k']+var_ranges['Max']['k'])/2   # [-]
fd = (var_ranges['Min']['Fractal Dimension']+var_ranges['Max']['Fractal Dimension'])/2   # [-]
ih = (var_ranges['Min']['Isosteric heat']+var_ranges['Max']['Isosteric heat'])/2   # [J/mol]
lp0 = (var_ranges['Min']['lp0']+var_ranges['Max']['lp0'])/2   # [Pa]
a0 = (var_ranges['Min']['a0']+var_ranges['Max']['a0'])/2   # [-]
a1 = (var_ranges['Min']['a1']+var_ranges['Max']['a1'])/2   # [-]
beta = (var_ranges['Min']['beta']+var_ranges['Max']['beta'])/2   # [-]
T = (var_ranges['Min']['T']+var_ranges['Max']['T'])/2   # [K]

In [18]:
def wu(p,ro,por,tor,op,q,t,k,fd,ih,lp0,a0,a1,beta,T):
    Z = interpolate.bisplev(p,T,f_z)   # [-]
    vis = interpolate.bisplev(p,T,f_vis)   # [Pa s]
    cg = f_cg(p)   # [1/MPa]
    dzdp = (interpolate.bisplev(p+10000,T,f_z)-interpolate.bisplev(p-10000,T,f_z))/20000

    porosity = por*((op-p)/athm_pr)**-q         # [-]
    radius = ro*((op-p)/athm_pr)**-t            # [m]
    kn = (vis/(p*2*radius))*(math.pi*Z*R*T/(2*M))**0.5   # [-]
    wv = 1/(1+kn)   # [-]
    wk = 1-wv   # [-]
    lp = lp0 * math.exp(-ih/(R*T))   # [Pa]
    theta = (p/Z)/(lp+p/Z)   # [-]
    rad = theta*dm   # [m]
    fc1 = (porosity/tor)*(1-rad/radius)**2   # [-]
    fc2 = fc1 * (((1-rad/radius)**-2)-1)   # [-]
    alpha = a0*2*math.atan(a1*kn**beta)/math.pi   # [-]
    B1 = wv*fc1*radius**2*p*M*(1+alpha*kn)*(1+4*kn/(1+kn))/(8*vis*Z*R*T)   # [s]
    B2 = wk*fc1*2*radius*(dm/radius)**(fd-2)*((8*Z*M/(math.pi*R*T))**0.5)*p*cg/(3*Z)   # [s]
    if k>=1:
        gam = 0
    else:
        gam = 1 
    ds0 = 8.29E-7 * T**0.5 * math.exp(-ih**0.8/(R*T))    # [m2/s]
    ds = ds0 * ((1-theta) + 0.5*k*theta*(2-theta) + gam*0.5*k*(1-k)*(theta**2))/((1-theta+0.5*k*theta)**2)    # [m2/s]
    Csc = 4*theta*M/(math.pi*dm**3*Na)    # [kg/m3]
    B3 = fc2*ds*Csc/(p)   # [s]
    D1 = (B1*R/M)/(1/(T*Z)+p*dzdp/(T*Z**2))   # [m2/s]
    D2 = (B2*R/M)/(1/(T*Z)+p*dzdp/(T*Z**2))   # [m2/s]
    D3 = (B3*R/M)/(1/(T*Z)+p*dzdp/(T*Z**2))   # [m2/s]
    DT = D1+D2+D3   # [m2/s]

    J = (B1+B2+B3)*dpdl*86400*365/1000

    return B1, B2, B3, J, DT, kn, dzdp, Z, cg, vis,  
